# Valuation Agent - Comparable Analysis Tool

An algorithmic valuation tool that estimates appropriate P/E and P/FCF multiples for a target company at 2-year and 5-year horizons using regression-based comparable analysis.

---

## Section 1: Configuration & Inputs

In [ ]:
# Cell 1.1: Imports & Setup

import bql
import pandas as pd
import numpy as np
from scipy import stats
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Initialize BQL connection
bq = bql.Service()

print("✓ BQL connection established")
print("✓ All libraries loaded successfully")

In [ ]:
# Cell 1.2: User Configuration

# =============================================================================
# CONFIGURATION PARAMETERS - Adjust these as needed
# =============================================================================

# GICS Level for sector peer selection
# Options: 'Sub_Industry', 'Industry', 'Industry_Group', 'Sector'
GICS_LEVEL = 'Sub_Industry'

# Market cap range multiplier relative to target company
MCAP_RANGE_LOW = 0.25   # Minimum market cap as multiple of target
MCAP_RANGE_HIGH = 4.0   # Maximum market cap as multiple of target

# Growth tolerance for growth cohort peers (percentage points)
GROWTH_TOLERANCE = 5.0  # ±5 percentage points

# Profitability tolerances
GROSS_MARGIN_TOLERANCE = 10.0   # ±10 percentage points
EBITDA_MARGIN_TOLERANCE = 5.0   # ±5 percentage points

# Minimum peer thresholds
MIN_PEERS = 10
MAX_PEERS = 25

# Mature comparables criteria (for 5-year terminal analysis)
MATURE_MIN_HISTORY_YEARS = 10
MATURE_MIN_REVENUE_B = 5.0      # $5 billion minimum
MATURE_MAX_GROWTH = 15.0        # Growth < 15%

# Outlier detection
OUTLIER_STD_THRESHOLD = 2.0     # Exclude peers > 2 standard deviations

# Confidence interval
CONFIDENCE_LEVEL = 0.95

print("Configuration loaded:")
print(f"  • GICS Level: {GICS_LEVEL}")
print(f"  • Market Cap Range: {MCAP_RANGE_LOW}x - {MCAP_RANGE_HIGH}x")
print(f"  • Growth Tolerance: ±{GROWTH_TOLERANCE} pp")
print(f"  • Min/Max Peers: {MIN_PEERS}-{MAX_PEERS}")

In [ ]:
# Cell 1.3: Target Company Input

# =============================================================================
# TARGET COMPANY - Enter your ticker here
# =============================================================================

TARGET_TICKER = "AAPL US Equity"  # Bloomberg ticker format

# =============================================================================
# 5-YEAR PROJECTIONS - Copy-paste from spreadsheet (tab-separated)
# Format: Year | Revenue | EPS | FCF
# =============================================================================

# Option 1: Paste tab-separated data as a string
PROJECTIONS_TSV = """
Year	Revenue	EPS	FCF
Y1	400000	6.50	100000
Y2	420000	7.00	110000
Y3	445000	7.60	120000
Y4	470000	8.20	130000
Y5	500000	8.90	145000
"""

# Parse the projections
def parse_projections(tsv_string):
    """Parse tab-separated projections data."""
    lines = [l.strip() for l in tsv_string.strip().split('\n') if l.strip()]
    if len(lines) < 2:
        raise ValueError("Projections must have header and at least one data row")
    
    header = lines[0].split('\t')
    data = []
    for line in lines[1:]:
        values = line.split('\t')
        row = {}
        for i, col in enumerate(header):
            val = values[i] if i < len(values) else ''
            # Try to convert to float if numeric
            try:
                row[col] = float(val.replace(',', '')) if col != 'Year' else val
            except ValueError:
                row[col] = val
        data.append(row)
    
    return pd.DataFrame(data)

# Parse and display projections
projections_df = parse_projections(PROJECTIONS_TSV)

# Calculate growth rates from projections
def calculate_projection_growth(df):
    """Calculate YoY and CAGR from projections."""
    result = df.copy()
    
    for col in ['Revenue', 'EPS', 'FCF']:
        if col in df.columns:
            # YoY growth
            result[f'{col}_YoY'] = df[col].pct_change() * 100
    
    # Calculate 5-year CAGR (from Y1 to Y5)
    cagr = {}
    for col in ['Revenue', 'EPS', 'FCF']:
        if col in df.columns and len(df) >= 2:
            start_val = df[col].iloc[0]
            end_val = df[col].iloc[-1]
            years = len(df) - 1
            if start_val > 0 and end_val > 0:
                cagr[f'{col}_CAGR'] = ((end_val / start_val) ** (1/years) - 1) * 100
    
    return result, cagr

projections_with_growth, projection_cagr = calculate_projection_growth(projections_df)

print(f"Target Company: {TARGET_TICKER}")
print("\n5-Year Projections:")
display(projections_with_growth)
print("\nProjected CAGRs:")
for k, v in projection_cagr.items():
    print(f"  • {k}: {v:.1f}%")

---

## Section 2: Data Retrieval

In [ ]:
# Cell 2.1: Target Company Data from Bloomberg

def get_target_company_data(ticker):
    """Retrieve comprehensive data for the target company."""
    
    # Build BQL request for target company
    # For NTM estimates, use FA_PERIOD_OFFSET=1 with FA_ACT_EST_DATA='E'
    request = bql.Request(ticker, {
        'Name': bq.data.name(),
        'Price': bq.data.px_last(),
        'Market_Cap': bq.data.cur_mkt_cap(),
        'Market_Cap_USD': bq.data.cur_mkt_cap(currency='USD'),
        
        # GICS Classification
        'GICS_Sector': bq.data.gics_sector_name(),
        'GICS_Industry_Group': bq.data.gics_industry_group_name(),
        'GICS_Industry': bq.data.gics_industry_name(),
        'GICS_Sub_Industry': bq.data.gics_sub_industry_name(),
        
        # Current Valuation
        'PE_LTM': bq.data.pe_ratio(),
        'EV_EBITDA': bq.data.ev_to_t12m_ebitda(),
        
        # Forward Estimates (NTM) - use FA_PERIOD_OFFSET=1 and FA_ACT_EST_DATA='E'
        'EPS_NTM': bq.data.is_eps(fa_period_offset=1, fa_act_est_data='E'),
        'EPS_Growth_NTM': bq.data.eeps_nxt_yr() / bq.data.is_eps() - 1,  # YoY growth
        
        # Margins
        'Gross_Margin': bq.data.gross_margin(),
        'EBITDA_Margin': bq.data.ebitda_margin(),
        'Net_Margin': bq.data.net_income_margin(),
        
        # FCF
        'FCF_Yield': bq.data.free_cash_flow_yield(),
        'FCF_Per_Share': bq.data.cf_free_cash_flow_per_sh(),
    })
    
    response = bq.execute(request)
    
    # Convert to dictionary
    data = {}
    for item in response:
        df = item.df()
        if not df.empty:
            col_name = df.columns[0]
            data[item.name] = df[col_name].iloc[0]
    
    # Calculate NTM P/E from price and NTM EPS
    if data.get('Price') and data.get('EPS_NTM') and data.get('EPS_NTM') > 0:
        data['PE_NTM'] = data['Price'] / data['EPS_NTM']
    else:
        data['PE_NTM'] = None
    
    # Convert EPS growth to percentage
    if data.get('EPS_Growth_NTM'):
        data['EPS_Growth_NTM'] = data['EPS_Growth_NTM'] * 100
    
    return data

# Fetch target company data
print(f"Fetching data for {TARGET_TICKER}...")
target_data = get_target_company_data(TARGET_TICKER)

# Display target company summary
print(f"\n{'='*60}")
print(f"TARGET COMPANY: {target_data.get('Name', TARGET_TICKER)}")
print(f"{'='*60}")
print(f"\nPrice: ${target_data.get('Price', 'N/A'):,.2f}")
print(f"Market Cap: ${target_data.get('Market_Cap_USD', 0)/1e9:,.1f}B")
print(f"\nGICS Classification:")
print(f"  Sector: {target_data.get('GICS_Sector', 'N/A')}")
print(f"  Industry Group: {target_data.get('GICS_Industry_Group', 'N/A')}")
print(f"  Industry: {target_data.get('GICS_Industry', 'N/A')}")
print(f"  Sub-Industry: {target_data.get('GICS_Sub_Industry', 'N/A')}")
print(f"\nValuation:")
pe_ntm = target_data.get('PE_NTM')
print(f"  P/E (NTM): {pe_ntm:.1f}x" if pe_ntm else "  P/E (NTM): N/A")
ev_ebitda = target_data.get('EV_EBITDA')
print(f"  EV/EBITDA: {ev_ebitda:.1f}x" if ev_ebitda else "  EV/EBITDA: N/A")
print(f"\nGrowth & Profitability:")
eps_growth = target_data.get('EPS_Growth_NTM')
print(f"  EPS Growth (NTM): {eps_growth:.1f}%" if eps_growth else "  EPS Growth (NTM): N/A")
gross_margin = target_data.get('Gross_Margin')
print(f"  Gross Margin: {gross_margin:.1f}%" if gross_margin else "  Gross Margin: N/A")
ebitda_margin = target_data.get('EBITDA_Margin')
print(f"  EBITDA Margin: {ebitda_margin:.1f}%" if ebitda_margin else "  EBITDA Margin: N/A")

In [ ]:
# Cell 2.2: Peer Universe Construction

def build_peer_universe(target_data, gics_level='Sub_Industry'):
    """
    Build initial peer universe based on GICS classification.
    Returns US equities in the same GICS category with positive earnings.
    """
    
    # Get GICS filter value based on level
    gics_mapping = {
        'Sector': ('GICS_Sector', bq.data.gics_sector_name),
        'Industry_Group': ('GICS_Industry_Group', bq.data.gics_industry_group_name),
        'Industry': ('GICS_Industry', bq.data.gics_industry_name),
        'Sub_Industry': ('GICS_Sub_Industry', bq.data.gics_sub_industry_name),
    }
    
    gics_key, gics_func = gics_mapping.get(gics_level, gics_mapping['Sub_Industry'])
    target_gics = target_data.get(gics_key)
    
    if not target_gics:
        raise ValueError(f"Could not find {gics_key} for target company")
    
    # Market cap range
    target_mcap = target_data.get('Market_Cap_USD', 0)
    min_mcap = target_mcap * MCAP_RANGE_LOW
    max_mcap = target_mcap * MCAP_RANGE_HIGH
    
    print(f"Building universe for {gics_level}: {target_gics}")
    print(f"Market cap range: ${min_mcap/1e9:.1f}B - ${max_mcap/1e9:.1f}B")
    
    # Build universe query
    # Start with US active equities
    universe = bq.univ.equitiesuniv(['ACTIVE', 'PRIMARY'])
    
    # Apply filters
    universe = universe.filter(gics_func() == target_gics)
    universe = universe.filter(bq.data.eqy_prim_exch_shrt().in_(['UN', 'UW', 'UA', 'UP']))  # US exchanges
    universe = universe.filter(bq.data.cur_mkt_cap(currency='USD') >= min_mcap)
    universe = universe.filter(bq.data.cur_mkt_cap(currency='USD') <= max_mcap)
    universe = universe.filter(bq.data.is_eps() > 0)  # Positive LTM earnings
    
    return universe, target_gics

# Build the peer universe
peer_universe, target_gics_value = build_peer_universe(target_data, GICS_LEVEL)
print(f"\nPeer universe constructed using {GICS_LEVEL} level")

In [ ]:
# Cell 2.3: Peer Data Retrieval

def get_peer_data(universe):
    """
    Retrieve comprehensive data for all peers in the universe.
    """
    
    # For NTM estimates, use FA_PERIOD_OFFSET=1 with FA_ACT_EST_DATA='E'
    request = bql.Request(universe, {
        'Name': bq.data.name(),
        'Ticker': bq.data.ticker(),
        
        # Valuation
        'PE_LTM': bq.data.pe_ratio(),
        
        # Market Data
        'Price': bq.data.px_last(),
        'Market_Cap': bq.data.cur_mkt_cap(currency='USD'),
        
        # Earnings - LTM and NTM estimates
        'EPS_LTM': bq.data.is_eps(),
        'EPS_NTM': bq.data.is_eps(fa_period_offset=1, fa_act_est_data='E'),
        'Revenue_Growth': bq.data.sales_growth(),
        
        # Profitability
        'Gross_Margin': bq.data.gross_margin(),
        'EBITDA_Margin': bq.data.ebitda_margin(),
        'Net_Margin': bq.data.net_income_margin(),
        'ROE': bq.data.return_com_eqy(),
        
        # FCF Metrics
        'FCF_Per_Share': bq.data.cf_free_cash_flow_per_sh(),
        'FCF_Yield': bq.data.free_cash_flow_yield(),
        
        # Historical (for 5-year analysis)
        'Years_Public': bq.data.security_days_trading() / 252,  # Approximate years
        'Revenue_5Y_CAGR': bq.data.sales_5yr_cagr(),
        'EPS_5Y_CAGR': bq.data.eps_5yr_cagr(),
    })
    
    response = bq.execute(request)
    
    # Combine all data into a single DataFrame
    dfs = []
    for item in response:
        df = item.df()
        if not df.empty:
            df = df.rename(columns={df.columns[0]: item.name})
            dfs.append(df)
    
    if not dfs:
        return pd.DataFrame()
    
    # Merge all dataframes on index
    result = dfs[0]
    for df in dfs[1:]:
        result = result.join(df, how='outer')
    
    # Calculate P/E NTM from price and NTM EPS
    if 'Price' in result.columns and 'EPS_NTM' in result.columns:
        result['PE_NTM'] = result['Price'] / result['EPS_NTM']
        result['PE_NTM'] = result['PE_NTM'].replace([np.inf, -np.inf], np.nan)
        # Filter out negative P/E
        result.loc[result['PE_NTM'] < 0, 'PE_NTM'] = np.nan
    
    # Calculate EPS Growth NTM
    if 'EPS_NTM' in result.columns and 'EPS_LTM' in result.columns:
        result['EPS_Growth_NTM'] = ((result['EPS_NTM'] / result['EPS_LTM']) - 1) * 100
        result['EPS_Growth_NTM'] = result['EPS_Growth_NTM'].replace([np.inf, -np.inf], np.nan)
    
    # Calculate P/FCF
    if 'Price' in result.columns and 'FCF_Per_Share' in result.columns:
        result['P_FCF'] = result['Price'] / result['FCF_Per_Share']
        result['P_FCF'] = result['P_FCF'].replace([np.inf, -np.inf], np.nan)
    
    return result

# Fetch peer data
print("Fetching peer data from Bloomberg...")
peers_df = get_peer_data(peer_universe)

# Remove target company from peers if present
target_ticker_clean = TARGET_TICKER.replace(' Equity', '').replace(' US', '')
if 'Ticker' in peers_df.columns:
    peers_df = peers_df[~peers_df['Ticker'].str.contains(target_ticker_clean, na=False)]

print(f"\nRetrieved data for {len(peers_df)} potential peers")
print(f"\nPeer Universe Summary:")
display(peers_df[['Name', 'Ticker', 'Market_Cap', 'PE_NTM', 'EPS_Growth_NTM', 'Gross_Margin']].head(10))

---

## Section 3: Peer Selection Logic

In [ ]:
# Cell 3.1: Sector Peers Selection

def select_sector_peers(peers_df, target_data, max_peers=MAX_PEERS):
    """
    Select sector peers based on GICS classification.
    Peers are already filtered by GICS in universe construction.
    Rank by similarity to target (market cap proximity).
    """
    
    df = peers_df.copy()
    
    # Filter for valid P/E
    df = df[df['PE_NTM'].notna() & (df['PE_NTM'] > 0) & (df['PE_NTM'] < 200)]
    
    # Filter for valid EPS growth
    df = df[df['EPS_Growth_NTM'].notna()]
    
    # Calculate similarity score based on market cap proximity
    target_mcap = target_data.get('Market_Cap_USD', 0)
    df['Mcap_Ratio'] = df['Market_Cap'] / target_mcap
    df['Mcap_Distance'] = abs(np.log(df['Mcap_Ratio']))  # Log distance from 1x
    
    # Sort by similarity (closest market cap first)
    df = df.sort_values('Mcap_Distance')
    
    # Select top peers
    selected = df.head(max_peers).copy()
    selected['Peer_Type'] = 'Sector'
    
    return selected

# Select sector peers
sector_peers = select_sector_peers(peers_df, target_data)
print(f"Selected {len(sector_peers)} sector peers")
print(f"\nSector Peers (ranked by market cap similarity):")
display(sector_peers[['Name', 'Ticker', 'Market_Cap', 'PE_NTM', 'EPS_Growth_NTM']].head(15))

In [ ]:
# Cell 3.2: Growth Cohort Peers (Cross-Sector)

def get_growth_cohort_universe(target_data):
    """
    Build universe for growth cohort peers (cross-sector).
    Filter by EPS growth within tolerance of target.
    """
    
    target_growth = target_data.get('EPS_Growth_NTM', 0)
    target_mcap = target_data.get('Market_Cap_USD', 0)
    
    min_growth = target_growth - GROWTH_TOLERANCE
    max_growth = target_growth + GROWTH_TOLERANCE
    min_mcap = target_mcap * MCAP_RANGE_LOW
    max_mcap = target_mcap * MCAP_RANGE_HIGH
    
    print(f"Target EPS Growth: {target_growth:.1f}%")
    print(f"Growth range: {min_growth:.1f}% - {max_growth:.1f}%")
    
    # Build cross-sector universe filtered by growth
    # Note: We'll filter by growth rate after fetching data since BQL 
    # doesn't support complex calculated filters in universe construction
    universe = bq.univ.equitiesuniv(['ACTIVE', 'PRIMARY'])
    universe = universe.filter(bq.data.eqy_prim_exch_shrt().in_(['UN', 'UW', 'UA', 'UP']))
    universe = universe.filter(bq.data.cur_mkt_cap(currency='USD') >= min_mcap)
    universe = universe.filter(bq.data.cur_mkt_cap(currency='USD') <= max_mcap)
    universe = universe.filter(bq.data.is_eps() > 0)  # Positive LTM earnings
    
    return universe, min_growth, max_growth

def select_growth_cohort_peers(target_data, max_peers=MAX_PEERS):
    """
    Select growth cohort peers from cross-sector universe.
    """
    
    # Get universe
    universe, min_growth, max_growth = get_growth_cohort_universe(target_data)
    
    # Fetch peer data
    print("Fetching growth cohort peers...")
    growth_peers_df = get_peer_data(universe)
    
    if growth_peers_df.empty:
        print("Warning: No growth cohort peers found")
        return pd.DataFrame()
    
    # Filter for valid data
    df = growth_peers_df.copy()
    df = df[df['PE_NTM'].notna() & (df['PE_NTM'] > 0) & (df['PE_NTM'] < 200)]
    df = df[df['EPS_Growth_NTM'].notna()]
    
    # Filter by growth range
    df = df[(df['EPS_Growth_NTM'] >= min_growth) & (df['EPS_Growth_NTM'] <= max_growth)]
    
    # Remove target company
    target_ticker_clean = TARGET_TICKER.replace(' Equity', '').replace(' US', '')
    if 'Ticker' in df.columns:
        df = df[~df['Ticker'].str.contains(target_ticker_clean, na=False)]
    
    # Calculate growth distance from target
    target_growth = target_data.get('EPS_Growth_NTM', 0)
    df['Growth_Distance'] = abs(df['EPS_Growth_NTM'] - target_growth)
    
    # Sort by growth similarity
    df = df.sort_values('Growth_Distance')
    
    # Select top peers
    selected = df.head(max_peers).copy()
    selected['Peer_Type'] = 'Growth'
    
    return selected

# Select growth cohort peers
growth_peers = select_growth_cohort_peers(target_data)
print(f"\nSelected {len(growth_peers)} growth cohort peers")
if not growth_peers.empty:
    print(f"\nGrowth Cohort Peers (cross-sector, ranked by growth similarity):")
    display(growth_peers[['Name', 'Ticker', 'Market_Cap', 'PE_NTM', 'EPS_Growth_NTM']].head(15))

In [ ]:
# Cell 3.3: Profitability Peers (Cross-Sector)

def get_profitability_universe(target_data):
    """
    Build universe for profitability peers (cross-sector).
    Filter by gross margin within tolerance of target.
    """
    
    target_gross_margin = target_data.get('Gross_Margin', 0)
    target_ebitda_margin = target_data.get('EBITDA_Margin', 0)
    target_mcap = target_data.get('Market_Cap_USD', 0)
    
    min_mcap = target_mcap * MCAP_RANGE_LOW
    max_mcap = target_mcap * MCAP_RANGE_HIGH
    
    # Use gross margin as primary filter
    min_gross = target_gross_margin - GROSS_MARGIN_TOLERANCE
    max_gross = target_gross_margin + GROSS_MARGIN_TOLERANCE
    
    print(f"Target Gross Margin: {target_gross_margin:.1f}%")
    print(f"Gross margin range: {min_gross:.1f}% - {max_gross:.1f}%")
    
    # Build cross-sector universe filtered by profitability
    universe = bq.univ.equitiesuniv(['ACTIVE', 'PRIMARY'])
    universe = universe.filter(bq.data.eqy_prim_exch_shrt().in_(['UN', 'UW', 'UA', 'UP']))
    universe = universe.filter(bq.data.cur_mkt_cap(currency='USD') >= min_mcap)
    universe = universe.filter(bq.data.cur_mkt_cap(currency='USD') <= max_mcap)
    universe = universe.filter(bq.data.gross_margin() >= min_gross)
    universe = universe.filter(bq.data.gross_margin() <= max_gross)
    universe = universe.filter(bq.data.is_eps() > 0)  # Positive LTM earnings
    
    return universe

def select_profitability_peers(target_data, max_peers=MAX_PEERS):
    """
    Select profitability peers from cross-sector universe.
    """
    
    # Get universe
    universe = get_profitability_universe(target_data)
    
    # Fetch peer data
    print("Fetching profitability peers...")
    profit_peers_df = get_peer_data(universe)
    
    if profit_peers_df.empty:
        print("Warning: No profitability peers found")
        return pd.DataFrame()
    
    # Filter for valid data
    df = profit_peers_df.copy()
    df = df[df['PE_NTM'].notna() & (df['PE_NTM'] > 0) & (df['PE_NTM'] < 200)]
    df = df[df['EPS_Growth_NTM'].notna()]
    
    # Remove target company
    target_ticker_clean = TARGET_TICKER.replace(' Equity', '').replace(' US', '')
    if 'Ticker' in df.columns:
        df = df[~df['Ticker'].str.contains(target_ticker_clean, na=False)]
    
    # Calculate profitability distance from target
    target_gross = target_data.get('Gross_Margin', 0)
    target_ebitda = target_data.get('EBITDA_Margin', 0)
    
    # Combined profitability score (normalized)
    df['Gross_Distance'] = abs(df['Gross_Margin'] - target_gross) / GROSS_MARGIN_TOLERANCE
    df['EBITDA_Distance'] = abs(df['EBITDA_Margin'].fillna(0) - target_ebitda) / EBITDA_MARGIN_TOLERANCE
    df['Profit_Distance'] = df['Gross_Distance'] + df['EBITDA_Distance'] * 0.5  # Weight gross margin more
    
    # Sort by profitability similarity
    df = df.sort_values('Profit_Distance')
    
    # Select top peers
    selected = df.head(max_peers).copy()
    selected['Peer_Type'] = 'Profitability'
    
    return selected

# Select profitability peers
profit_peers = select_profitability_peers(target_data)
print(f"\nSelected {len(profit_peers)} profitability peers")
if not profit_peers.empty:
    print(f"\nProfitability Peers (cross-sector, ranked by margin similarity):")
    display(profit_peers[['Name', 'Ticker', 'Market_Cap', 'PE_NTM', 'Gross_Margin', 'EBITDA_Margin']].head(15))

In [ ]:
# Cell 3.4: Mature Comparables (5-Year Terminal Analysis)

def get_mature_comps_universe(target_data):
    """
    Build universe for mature comparables.
    Same sector, long trading history, large cap, lower growth.
    """
    
    target_sector = target_data.get('GICS_Sector')
    
    print(f"Building mature comparables universe for sector: {target_sector}")
    print(f"Criteria: Revenue > ${MATURE_MIN_REVENUE_B}B, Growth < {MATURE_MAX_GROWTH}%")
    
    # Build universe of mature companies in same sector
    universe = bq.univ.equitiesuniv(['ACTIVE', 'PRIMARY'])
    universe = universe.filter(bq.data.gics_sector_name() == target_sector)
    universe = universe.filter(bq.data.eqy_prim_exch_shrt().in_(['UN', 'UW', 'UA', 'UP']))
    universe = universe.filter(bq.data.sales_rev_turn() >= MATURE_MIN_REVENUE_B * 1e9)  # Revenue > $5B
    universe = universe.filter(bq.data.is_eps() > 0)  # Positive LTM earnings
    
    return universe

def get_mature_peer_data(universe):
    """
    Retrieve data for mature comparables including historical averages.
    """
    
    request = bql.Request(universe, {
        'Name': bq.data.name(),
        'Ticker': bq.data.ticker(),
        
        # Current Valuation
        'PE_LTM': bq.data.pe_ratio(),
        
        # Market Data
        'Price': bq.data.px_last(),
        'Market_Cap': bq.data.cur_mkt_cap(currency='USD'),
        'Revenue': bq.data.sales_rev_turn(),
        
        # Earnings - LTM and NTM estimates
        'EPS_LTM': bq.data.is_eps(),
        'EPS_NTM': bq.data.is_eps(fa_period_offset=1, fa_act_est_data='E'),
        
        # Growth
        'EPS_5Y_CAGR': bq.data.eps_5yr_cagr(),
        'Revenue_5Y_CAGR': bq.data.sales_5yr_cagr(),
        
        # Profitability
        'Gross_Margin': bq.data.gross_margin(),
        'EBITDA_Margin': bq.data.ebitda_margin(),
        
        # FCF
        'FCF_Per_Share': bq.data.cf_free_cash_flow_per_sh(),
        
        # Historical averages (5-year)
        'PE_5Y_Avg': bq.data.pe_ratio().avg(dates=bq.func.range('-5Y', '0D')),
        
        # Trading history
        'Years_Public': bq.data.security_days_trading() / 252,
    })
    
    response = bq.execute(request)
    
    # Combine all data into a single DataFrame
    dfs = []
    for item in response:
        df = item.df()
        if not df.empty:
            df = df.rename(columns={df.columns[0]: item.name})
            dfs.append(df)
    
    if not dfs:
        return pd.DataFrame()
    
    # Merge all dataframes
    result = dfs[0]
    for df in dfs[1:]:
        result = result.join(df, how='outer')
    
    # Calculate P/E NTM from price and NTM EPS
    if 'Price' in result.columns and 'EPS_NTM' in result.columns:
        result['PE_NTM'] = result['Price'] / result['EPS_NTM']
        result['PE_NTM'] = result['PE_NTM'].replace([np.inf, -np.inf], np.nan)
        result.loc[result['PE_NTM'] < 0, 'PE_NTM'] = np.nan
    
    # Calculate EPS Growth NTM
    if 'EPS_NTM' in result.columns and 'EPS_LTM' in result.columns:
        result['EPS_Growth_NTM'] = ((result['EPS_NTM'] / result['EPS_LTM']) - 1) * 100
        result['EPS_Growth_NTM'] = result['EPS_Growth_NTM'].replace([np.inf, -np.inf], np.nan)
    
    # Calculate P/FCF
    if 'Price' in result.columns and 'FCF_Per_Share' in result.columns:
        result['P_FCF'] = result['Price'] / result['FCF_Per_Share']
        result['P_FCF'] = result['P_FCF'].replace([np.inf, -np.inf], np.nan)
    
    return result

def select_mature_comparables(target_data, min_peers=MIN_PEERS):
    """
    Select mature comparables for 5-year terminal multiple analysis.
    """
    
    # Get universe
    universe = get_mature_comps_universe(target_data)
    
    # Fetch peer data
    print("Fetching mature comparables...")
    mature_df = get_mature_peer_data(universe)
    
    if mature_df.empty:
        print("Warning: No mature comparables found")
        return pd.DataFrame()
    
    # Filter for valid data
    df = mature_df.copy()
    df = df[df['PE_NTM'].notna() & (df['PE_NTM'] > 0) & (df['PE_NTM'] < 100)]
    df = df[df['EPS_5Y_CAGR'].notna()]
    df = df[df['Years_Public'] >= MATURE_MIN_HISTORY_YEARS]  # 10+ years trading
    
    # Filter for lower growth (mature companies)
    if 'EPS_Growth_NTM' in df.columns:
        df = df[df['EPS_Growth_NTM'] < MATURE_MAX_GROWTH]
    
    # Remove target company
    target_ticker_clean = TARGET_TICKER.replace(' Equity', '').replace(' US', '')
    if 'Ticker' in df.columns:
        df = df[~df['Ticker'].str.contains(target_ticker_clean, na=False)]
    
    # Sort by market cap (largest first for mature comps)
    df = df.sort_values('Market_Cap', ascending=False)
    
    # Select peers
    selected = df.copy()
    selected['Peer_Type'] = 'Mature'
    
    if len(selected) < min_peers:
        print(f"Warning: Only {len(selected)} mature comparables found (minimum: {min_peers})")
    
    return selected

# Select mature comparables
mature_peers = select_mature_comparables(target_data)
print(f"\nSelected {len(mature_peers)} mature comparables")
if not mature_peers.empty:
    print(f"\nMature Comparables (for 5-year terminal analysis):")
    display(mature_peers[['Name', 'Ticker', 'Market_Cap', 'PE_NTM', 'PE_5Y_Avg', 'EPS_5Y_CAGR']].head(15))

---

## Section 4: Regression Analysis

In [ ]:
# Cell 4.1: Core Regression Functions

def remove_outliers(df, x_col, y_col, std_threshold=OUTLIER_STD_THRESHOLD):
    """
    Remove outliers that are > std_threshold standard deviations from regression line.
    Returns filtered dataframe and outlier dataframe.
    """
    
    # Initial regression
    valid_mask = df[x_col].notna() & df[y_col].notna()
    valid_df = df[valid_mask].copy()
    
    if len(valid_df) < 3:
        return valid_df, pd.DataFrame()
    
    x = valid_df[x_col].values
    y = valid_df[y_col].values
    
    # Fit regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    
    # Calculate residuals
    predicted = slope * x + intercept
    residuals = y - predicted
    residual_std = np.std(residuals)
    
    # Identify outliers
    outlier_mask = np.abs(residuals) > std_threshold * residual_std
    
    # Return filtered and outlier dataframes
    valid_df['Is_Outlier'] = outlier_mask
    filtered_df = valid_df[~outlier_mask].copy()
    outlier_df = valid_df[outlier_mask].copy()
    
    return filtered_df, outlier_df


def run_regression(df, x_col, y_col, target_x_value, remove_outliers_flag=True):
    """
    Run OLS regression and calculate implied Y value for target.
    
    Returns:
        dict with regression results, confidence intervals, and implied value
    """
    
    # Remove outliers if requested
    if remove_outliers_flag:
        filtered_df, outlier_df = remove_outliers(df, x_col, y_col)
    else:
        valid_mask = df[x_col].notna() & df[y_col].notna()
        filtered_df = df[valid_mask].copy()
        outlier_df = pd.DataFrame()
    
    if len(filtered_df) < 3:
        return {
            'success': False,
            'error': 'Insufficient data points for regression',
            'peer_count': len(filtered_df)
        }
    
    x = filtered_df[x_col].values
    y = filtered_df[y_col].values
    n = len(x)
    
    # Run regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    r_squared = r_value ** 2
    
    # Calculate implied Y value for target
    implied_y = slope * target_x_value + intercept
    
    # Calculate prediction interval for target
    x_mean = np.mean(x)
    ss_xx = np.sum((x - x_mean) ** 2)
    
    # Residual standard error
    y_pred = slope * x + intercept
    residuals = y - y_pred
    mse = np.sum(residuals ** 2) / (n - 2)
    se_residual = np.sqrt(mse)
    
    # Standard error of prediction for new point
    se_pred = se_residual * np.sqrt(1 + 1/n + (target_x_value - x_mean)**2 / ss_xx)
    
    # Confidence interval (t-distribution)
    t_value = stats.t.ppf((1 + CONFIDENCE_LEVEL) / 2, n - 2)
    ci_lower = implied_y - t_value * se_pred
    ci_upper = implied_y + t_value * se_pred
    
    # Check if target is within peer range (extrapolation warning)
    x_min, x_max = np.min(x), np.max(x)
    is_extrapolation = target_x_value < x_min or target_x_value > x_max
    
    return {
        'success': True,
        'slope': slope,
        'intercept': intercept,
        'r_squared': r_squared,
        'p_value': p_value,
        'std_error': std_err,
        'implied_value': implied_y,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'peer_count': n,
        'outliers_removed': len(outlier_df),
        'is_extrapolation': is_extrapolation,
        'x_range': (x_min, x_max),
        'peers_df': filtered_df,
        'outliers_df': outlier_df,
        'x_col': x_col,
        'y_col': y_col,
        'target_x': target_x_value
    }


def generate_warnings(regression_result, dimension_name):
    """Generate warnings based on regression results."""
    
    warnings = []
    
    if not regression_result.get('success', False):
        warnings.append(f"⚠️ {dimension_name}: {regression_result.get('error', 'Unknown error')}")
        return warnings
    
    # Peer count warning
    if regression_result['peer_count'] < MIN_PEERS:
        warnings.append(f"⚠️ {dimension_name}: Small peer set ({regression_result['peer_count']} peers). Consider widening criteria.")
    
    # R-squared warning
    if regression_result['r_squared'] < 0.4:
        warnings.append(f"⚠️ {dimension_name}: Weak regression fit (R²={regression_result['r_squared']:.2f}). Interpret with caution.")
    
    # Extrapolation warning
    if regression_result['is_extrapolation']:
        x_min, x_max = regression_result['x_range']
        warnings.append(f"⚠️ {dimension_name}: Target outside peer range ({x_min:.1f}% - {x_max:.1f}%). Multiple capped at boundary.")
    
    return warnings

print("Regression functions defined successfully")

In [ ]:
# Cell 4.2: 2-Year Regressions (P/E and P/FCF)

# Store all regression results
regression_results = {}
all_warnings = []

# Get target's growth rate for regression
target_eps_growth = target_data.get('EPS_Growth_NTM', 0)

print(f"Target EPS Growth (NTM): {target_eps_growth:.1f}%")
print(f"\n{'='*60}")
print("RUNNING 2-YEAR P/E REGRESSIONS")
print(f"{'='*60}\n")

# 2-Year P/E Regressions for each dimension
pe_dimensions = {
    'Sector': sector_peers,
    'Growth': growth_peers,
    'Profitability': profit_peers
}

for dim_name, peers_df in pe_dimensions.items():
    print(f"\n--- {dim_name} Peers P/E Regression ---")
    
    if peers_df.empty:
        print(f"No peers available for {dim_name} dimension")
        regression_results[f'PE_2Y_{dim_name}'] = {'success': False, 'error': 'No peers available'}
        continue
    
    result = run_regression(
        df=peers_df,
        x_col='EPS_Growth_NTM',
        y_col='PE_NTM',
        target_x_value=target_eps_growth
    )
    
    regression_results[f'PE_2Y_{dim_name}'] = result
    
    if result['success']:
        print(f"  Peers: {result['peer_count']} (removed {result['outliers_removed']} outliers)")
        print(f"  R²: {result['r_squared']:.3f}")
        print(f"  Equation: P/E = {result['slope']:.2f} × Growth + {result['intercept']:.1f}")
        print(f"  Implied P/E: {result['implied_value']:.1f}x")
        print(f"  95% CI: {result['ci_lower']:.1f}x - {result['ci_upper']:.1f}x")
        
        # Generate warnings
        warnings = generate_warnings(result, f"P/E {dim_name}")
        all_warnings.extend(warnings)
    else:
        print(f"  Error: {result.get('error', 'Unknown')}")

# 2-Year P/FCF Regressions (parallel)
print(f"\n{'='*60}")
print("RUNNING 2-YEAR P/FCF REGRESSIONS")
print(f"{'='*60}\n")

for dim_name, peers_df in pe_dimensions.items():
    print(f"\n--- {dim_name} Peers P/FCF Regression ---")
    
    if peers_df.empty or 'P_FCF' not in peers_df.columns:
        print(f"No P/FCF data available for {dim_name} dimension")
        regression_results[f'PFCF_2Y_{dim_name}'] = {'success': False, 'error': 'No P/FCF data'}
        continue
    
    # Filter for valid P/FCF (positive and reasonable)
    valid_pfcf = peers_df[(peers_df['P_FCF'].notna()) & 
                          (peers_df['P_FCF'] > 0) & 
                          (peers_df['P_FCF'] < 200)].copy()
    
    if len(valid_pfcf) < 5:
        print(f"Insufficient P/FCF data for {dim_name} ({len(valid_pfcf)} peers)")
        regression_results[f'PFCF_2Y_{dim_name}'] = {'success': False, 'error': 'Insufficient P/FCF data'}
        continue
    
    result = run_regression(
        df=valid_pfcf,
        x_col='EPS_Growth_NTM',
        y_col='P_FCF',
        target_x_value=target_eps_growth
    )
    
    regression_results[f'PFCF_2Y_{dim_name}'] = result
    
    if result['success']:
        print(f"  Peers: {result['peer_count']} (removed {result['outliers_removed']} outliers)")
        print(f"  R²: {result['r_squared']:.3f}")
        print(f"  Equation: P/FCF = {result['slope']:.2f} × Growth + {result['intercept']:.1f}")
        print(f"  Implied P/FCF: {result['implied_value']:.1f}x")
        print(f"  95% CI: {result['ci_lower']:.1f}x - {result['ci_upper']:.1f}x")
        
        warnings = generate_warnings(result, f"P/FCF {dim_name}")
        all_warnings.extend(warnings)
    else:
        print(f"  Error: {result.get('error', 'Unknown')}")

In [ ]:
# Cell 4.3: 5-Year Terminal Multiple Regressions

print(f"\n{'='*60}")
print("RUNNING 5-YEAR TERMINAL MULTIPLE REGRESSIONS")
print(f"{'='*60}\n")

# Get projected Year 5 growth rate from user projections
target_y5_eps_cagr = projection_cagr.get('EPS_CAGR', 0)
target_y5_fcf_cagr = projection_cagr.get('FCF_CAGR', 0)

print(f"Target 5-Year Projected EPS CAGR: {target_y5_eps_cagr:.1f}%")
print(f"Target 5-Year Projected FCF CAGR: {target_y5_fcf_cagr:.1f}%")

# 5-Year P/E Regression using mature comparables
print(f"\n--- 5-Year Terminal P/E Regression (Mature Comps) ---")

if mature_peers.empty:
    print("No mature comparables available")
    regression_results['PE_5Y_Terminal'] = {'success': False, 'error': 'No mature comparables'}
else:
    # Filter for valid historical P/E average
    valid_mature = mature_peers[
        (mature_peers['PE_5Y_Avg'].notna()) & 
        (mature_peers['PE_5Y_Avg'] > 0) & 
        (mature_peers['PE_5Y_Avg'] < 100) &
        (mature_peers['EPS_5Y_CAGR'].notna())
    ].copy()
    
    if len(valid_mature) < 5:
        print(f"Insufficient data for terminal P/E regression ({len(valid_mature)} peers)")
        regression_results['PE_5Y_Terminal'] = {'success': False, 'error': 'Insufficient historical data'}
    else:
        result = run_regression(
            df=valid_mature,
            x_col='EPS_5Y_CAGR',
            y_col='PE_5Y_Avg',  # Historical 5-year average P/E
            target_x_value=target_y5_eps_cagr
        )
        
        regression_results['PE_5Y_Terminal'] = result
        
        if result['success']:
            print(f"  Peers: {result['peer_count']} (removed {result['outliers_removed']} outliers)")
            print(f"  R²: {result['r_squared']:.3f}")
            print(f"  Equation: P/E(5Y Avg) = {result['slope']:.2f} × 5Y CAGR + {result['intercept']:.1f}")
            print(f"  Implied Terminal P/E: {result['implied_value']:.1f}x")
            print(f"  95% CI: {result['ci_lower']:.1f}x - {result['ci_upper']:.1f}x")
            
            warnings = generate_warnings(result, "5Y Terminal P/E")
            all_warnings.extend(warnings)
        else:
            print(f"  Error: {result.get('error', 'Unknown')}")

# 5-Year P/FCF Regression using mature comparables
print(f"\n--- 5-Year Terminal P/FCF Regression (Mature Comps) ---")

if mature_peers.empty:
    print("No mature comparables available")
    regression_results['PFCF_5Y_Terminal'] = {'success': False, 'error': 'No mature comparables'}
else:
    # Filter for valid P/FCF data
    valid_pfcf_mature = mature_peers[
        (mature_peers['P_FCF'].notna()) & 
        (mature_peers['P_FCF'] > 0) & 
        (mature_peers['P_FCF'] < 100) &
        (mature_peers['EPS_5Y_CAGR'].notna())
    ].copy()
    
    if len(valid_pfcf_mature) < 5:
        print(f"Insufficient data for terminal P/FCF regression ({len(valid_pfcf_mature)} peers)")
        regression_results['PFCF_5Y_Terminal'] = {'success': False, 'error': 'Insufficient P/FCF data'}
    else:
        result = run_regression(
            df=valid_pfcf_mature,
            x_col='EPS_5Y_CAGR',
            y_col='P_FCF',
            target_x_value=target_y5_fcf_cagr
        )
        
        regression_results['PFCF_5Y_Terminal'] = result
        
        if result['success']:
            print(f"  Peers: {result['peer_count']} (removed {result['outliers_removed']} outliers)")
            print(f"  R²: {result['r_squared']:.3f}")
            print(f"  Equation: P/FCF = {result['slope']:.2f} × 5Y CAGR + {result['intercept']:.1f}")
            print(f"  Implied Terminal P/FCF: {result['implied_value']:.1f}x")
            print(f"  95% CI: {result['ci_lower']:.1f}x - {result['ci_upper']:.1f}x")
            
            warnings = generate_warnings(result, "5Y Terminal P/FCF")
            all_warnings.extend(warnings)
        else:
            print(f"  Error: {result.get('error', 'Unknown')}")

# Display all warnings
if all_warnings:
    print(f"\n{'='*60}")
    print("WARNINGS")
    print(f"{'='*60}")
    for w in all_warnings:
        print(w)

---

## Section 5: Interactive Visualizations

In [ ]:
# Cell 5.1: Regression Scatter Plot Function

def create_regression_scatter(result, title, x_label, y_label, target_name="Target"):
    """
    Create an interactive Plotly scatter plot with regression line and confidence band.
    """
    
    if not result.get('success', False):
        return None
    
    peers_df = result['peers_df']
    outliers_df = result.get('outliers_df', pd.DataFrame())
    
    x_col = result['x_col']
    y_col = result['y_col']
    
    fig = go.Figure()
    
    # Get x range for regression line
    x_vals = peers_df[x_col].values
    x_min, x_max = np.min(x_vals), np.max(x_vals)
    x_range = np.linspace(x_min - 2, x_max + 2, 100)
    
    # Calculate regression line and confidence band
    slope = result['slope']
    intercept = result['intercept']
    y_line = slope * x_range + intercept
    
    # Calculate confidence band (simplified - using uniform width based on R²)
    se = (1 - result['r_squared']) * np.std(peers_df[y_col].values) * 1.96
    y_upper = y_line + se
    y_lower = y_line - se
    
    # Add confidence band
    fig.add_trace(go.Scatter(
        x=np.concatenate([x_range, x_range[::-1]]),
        y=np.concatenate([y_upper, y_lower[::-1]]),
        fill='toself',
        fillcolor='rgba(100, 149, 237, 0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        showlegend=True,
        name='95% Confidence Band',
        hoverinfo='skip'
    ))
    
    # Add regression line
    fig.add_trace(go.Scatter(
        x=x_range,
        y=y_line,
        mode='lines',
        line=dict(color='blue', width=2),
        name=f'Regression (R²={result["r_squared"]:.2f})',
        hovertemplate=f'{y_label}: %{{y:.1f}}x<br>{x_label}: %{{x:.1f}}%<extra></extra>'
    ))
    
    # Add peer points
    hover_text = [f"{row.get('Ticker', row.name)}<br>{row.get('Name', '')}<br>"
                  f"{y_label}: {row[y_col]:.1f}x<br>{x_label}: {row[x_col]:.1f}%"
                  for _, row in peers_df.iterrows()]
    
    fig.add_trace(go.Scatter(
        x=peers_df[x_col],
        y=peers_df[y_col],
        mode='markers',
        marker=dict(size=10, color='cornflowerblue', line=dict(width=1, color='darkblue')),
        name=f'Peers (n={len(peers_df)})',
        text=hover_text,
        hovertemplate='%{text}<extra></extra>',
        customdata=peers_df.index.tolist()
    ))
    
    # Add outliers (if any)
    if not outliers_df.empty:
        outlier_text = [f"{row.get('Ticker', row.name)}<br>{row.get('Name', '')} (Outlier)<br>"
                       f"{y_label}: {row[y_col]:.1f}x<br>{x_label}: {row[x_col]:.1f}%"
                       for _, row in outliers_df.iterrows()]
        
        fig.add_trace(go.Scatter(
            x=outliers_df[x_col],
            y=outliers_df[y_col],
            mode='markers',
            marker=dict(size=8, color='lightgray', symbol='x', 
                       line=dict(width=1, color='gray')),
            name=f'Excluded Outliers (n={len(outliers_df)})',
            text=outlier_text,
            hovertemplate='%{text}<extra></extra>'
        ))
    
    # Add target company point
    target_x = result['target_x']
    implied_y = result['implied_value']
    
    fig.add_trace(go.Scatter(
        x=[target_x],
        y=[implied_y],
        mode='markers',
        marker=dict(size=15, color='red', symbol='star', 
                   line=dict(width=2, color='darkred')),
        name=f'{target_name} (Implied: {implied_y:.1f}x)',
        hovertemplate=f'{target_name}<br>{y_label}: {implied_y:.1f}x<br>{x_label}: {target_x:.1f}%<extra></extra>'
    ))
    
    # Add confidence interval annotation
    ci_lower = result['ci_lower']
    ci_upper = result['ci_upper']
    
    # Update layout
    fig.update_layout(
        title=dict(
            text=f"{title}<br><sup>Implied {y_label}: {implied_y:.1f}x (95% CI: {ci_lower:.1f}x - {ci_upper:.1f}x)</sup>",
            x=0.5,
            xanchor='center'
        ),
        xaxis_title=x_label,
        yaxis_title=y_label,
        hovermode='closest',
        showlegend=True,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ),
        height=500,
        template='plotly_white'
    )
    
    return fig

print("Scatter plot function defined")

In [ ]:
# Cell 5.2: 2-Year P/E Regression Charts

target_name = target_data.get('Name', TARGET_TICKER)

print("2-Year P/E Regression Charts")
print("="*60)

# Sector Peers P/E
if regression_results.get('PE_2Y_Sector', {}).get('success'):
    fig_sector = create_regression_scatter(
        regression_results['PE_2Y_Sector'],
        title=f"P/E vs EPS Growth: Sector Peers ({target_gics_value})",
        x_label="NTM EPS Growth (%)",
        y_label="P/E (NTM)",
        target_name=target_name
    )
    if fig_sector:
        fig_sector.show()

# Growth Cohort P/E
if regression_results.get('PE_2Y_Growth', {}).get('success'):
    fig_growth = create_regression_scatter(
        regression_results['PE_2Y_Growth'],
        title="P/E vs EPS Growth: Growth Cohort Peers (Cross-Sector)",
        x_label="NTM EPS Growth (%)",
        y_label="P/E (NTM)",
        target_name=target_name
    )
    if fig_growth:
        fig_growth.show()

# Profitability Peers P/E
if regression_results.get('PE_2Y_Profitability', {}).get('success'):
    fig_profit = create_regression_scatter(
        regression_results['PE_2Y_Profitability'],
        title="P/E vs EPS Growth: Profitability Peers (Cross-Sector)",
        x_label="NTM EPS Growth (%)",
        y_label="P/E (NTM)",
        target_name=target_name
    )
    if fig_profit:
        fig_profit.show()

In [ ]:
# Cell 5.3: 5-Year Terminal Multiple Chart

print("5-Year Terminal P/E Regression Chart")
print("="*60)

# 5-Year Terminal P/E
if regression_results.get('PE_5Y_Terminal', {}).get('success'):
    fig_terminal = create_regression_scatter(
        regression_results['PE_5Y_Terminal'],
        title="Terminal P/E: Mature Comparables (5-Year Historical Avg)",
        x_label="5-Year EPS CAGR (%)",
        y_label="5-Year Avg P/E",
        target_name=target_name
    )
    if fig_terminal:
        fig_terminal.show()
else:
    print("Terminal P/E regression not available")

In [ ]:
# Cell 5.4: Summary Comparison Chart

def create_summary_chart(regression_results, metric='PE'):
    """
    Create a bar chart comparing implied multiples across all dimensions.
    """
    
    # Collect results for the specified metric
    data = []
    
    if metric == 'PE':
        result_keys = [
            ('PE_2Y_Sector', '2Y Sector', '#1f77b4'),
            ('PE_2Y_Growth', '2Y Growth', '#2ca02c'),
            ('PE_2Y_Profitability', '2Y Profitability', '#9467bd'),
            ('PE_5Y_Terminal', '5Y Terminal', '#d62728'),
        ]
        title = 'Implied P/E Multiple Comparison'
        y_label = 'Implied P/E'
    else:  # P/FCF
        result_keys = [
            ('PFCF_2Y_Sector', '2Y Sector', '#1f77b4'),
            ('PFCF_2Y_Growth', '2Y Growth', '#2ca02c'),
            ('PFCF_2Y_Profitability', '2Y Profitability', '#9467bd'),
            ('PFCF_5Y_Terminal', '5Y Terminal', '#d62728'),
        ]
        title = 'Implied P/FCF Multiple Comparison'
        y_label = 'Implied P/FCF'
    
    for key, label, color in result_keys:
        result = regression_results.get(key, {})
        if result.get('success'):
            data.append({
                'Dimension': label,
                'Implied': result['implied_value'],
                'CI_Lower': result['ci_lower'],
                'CI_Upper': result['ci_upper'],
                'R_Squared': result['r_squared'],
                'Peer_Count': result['peer_count'],
                'Color': color
            })
    
    if not data:
        return None
    
    df = pd.DataFrame(data)
    
    # Create bar chart with error bars
    fig = go.Figure()
    
    # Add bars
    fig.add_trace(go.Bar(
        x=df['Dimension'],
        y=df['Implied'],
        marker_color=df['Color'],
        error_y=dict(
            type='data',
            symmetric=False,
            array=df['CI_Upper'] - df['Implied'],
            arrayminus=df['Implied'] - df['CI_Lower'],
            color='rgba(0,0,0,0.3)'
        ),
        text=[f"{v:.1f}x" for v in df['Implied']],
        textposition='outside',
        hovertemplate=(
            '%{x}<br>' +
            'Implied: %{y:.1f}x<br>' +
            'CI: %{customdata[0]:.1f}x - %{customdata[1]:.1f}x<br>' +
            'R²: %{customdata[2]:.2f}<br>' +
            'Peers: %{customdata[3]}<extra></extra>'
        ),
        customdata=df[['CI_Lower', 'CI_Upper', 'R_Squared', 'Peer_Count']].values
    ))
    
    # Add current P/E reference line
    current_pe = target_data.get('PE_NTM', None)
    if current_pe and metric == 'PE':
        fig.add_hline(
            y=current_pe,
            line_dash="dash",
            line_color="gray",
            annotation_text=f"Current P/E: {current_pe:.1f}x",
            annotation_position="top right"
        )
    
    fig.update_layout(
        title=dict(text=title, x=0.5, xanchor='center'),
        xaxis_title="Analysis Dimension",
        yaxis_title=y_label,
        height=450,
        template='plotly_white',
        showlegend=False
    )
    
    return fig

# Create P/E summary chart
print("Summary Comparison Charts")
print("="*60)

fig_pe_summary = create_summary_chart(regression_results, metric='PE')
if fig_pe_summary:
    fig_pe_summary.show()

# Create P/FCF summary chart  
fig_pfcf_summary = create_summary_chart(regression_results, metric='PFCF')
if fig_pfcf_summary:
    fig_pfcf_summary.show()

---

## Section 6: Results Output

In [ ]:
# Cell 6.1: Summary Tables

def build_summary_table(regression_results, metric='PE'):
    """Build summary table for P/E or P/FCF results."""
    
    if metric == 'PE':
        rows = [
            ('2-Year', 'Sector', 'PE_2Y_Sector'),
            ('2-Year', 'Growth', 'PE_2Y_Growth'),
            ('2-Year', 'Profitability', 'PE_2Y_Profitability'),
            ('5-Year', 'Mature Comps', 'PE_5Y_Terminal'),
        ]
        metric_name = 'P/E'
    else:
        rows = [
            ('2-Year', 'Sector', 'PFCF_2Y_Sector'),
            ('2-Year', 'Growth', 'PFCF_2Y_Growth'),
            ('2-Year', 'Profitability', 'PFCF_2Y_Profitability'),
            ('5-Year', 'Mature Comps', 'PFCF_5Y_Terminal'),
        ]
        metric_name = 'P/FCF'
    
    data = []
    for horizon, dimension, key in rows:
        result = regression_results.get(key, {})
        
        if result.get('success'):
            data.append({
                'Horizon': horizon,
                'Dimension': dimension,
                f'Implied {metric_name}': f"{result['implied_value']:.1f}x",
                'Confidence Range': f"{result['ci_lower']:.1f}x - {result['ci_upper']:.1f}x",
                'R²': f"{result['r_squared']:.2f}",
                'Peer Count': result['peer_count'],
                'Notes': 'Terminal' if '5Y' in key else '—'
            })
        else:
            data.append({
                'Horizon': horizon,
                'Dimension': dimension,
                f'Implied {metric_name}': 'N/A',
                'Confidence Range': 'N/A',
                'R²': 'N/A',
                'Peer Count': 0,
                'Notes': result.get('error', 'Error')
            })
    
    return pd.DataFrame(data)

# Build and display P/E summary table
print(f"\n{'='*80}")
print(f"SUMMARY TABLE: P/E MULTIPLES")
print(f"{'='*80}\n")

pe_summary_df = build_summary_table(regression_results, metric='PE')
display(pe_summary_df.style.set_properties(**{'text-align': 'center'}))

# Build and display P/FCF summary table
print(f"\n{'='*80}")
print(f"SUMMARY TABLE: P/FCF MULTIPLES")
print(f"{'='*80}\n")

pfcf_summary_df = build_summary_table(regression_results, metric='PFCF')
display(pfcf_summary_df.style.set_properties(**{'text-align': 'center'}))

In [ ]:
# Cell 6.2: Peer Detail Tables

def display_peer_details(result, dimension_name):
    """Display detailed peer table for a regression result."""
    
    if not result.get('success'):
        print(f"{dimension_name}: No data available")
        return
    
    peers_df = result['peers_df']
    outliers_df = result.get('outliers_df', pd.DataFrame())
    
    # Prepare display columns
    display_cols = ['Ticker', 'Name', 'Market_Cap', 'PE_NTM', 'EPS_Growth_NTM', 'Gross_Margin']
    available_cols = [c for c in display_cols if c in peers_df.columns]
    
    # Add status column
    peers_display = peers_df[available_cols].copy()
    peers_display['Status'] = 'Included'
    
    # Add outliers if any
    if not outliers_df.empty:
        outliers_display = outliers_df[available_cols].copy()
        outliers_display['Status'] = 'Excluded (Outlier)'
        peers_display = pd.concat([peers_display, outliers_display])
    
    # Format numeric columns
    if 'Market_Cap' in peers_display.columns:
        peers_display['Market_Cap'] = peers_display['Market_Cap'].apply(
            lambda x: f"${x/1e9:.1f}B" if pd.notna(x) else 'N/A'
        )
    if 'PE_NTM' in peers_display.columns:
        peers_display['PE_NTM'] = peers_display['PE_NTM'].apply(
            lambda x: f"{x:.1f}x" if pd.notna(x) else 'N/A'
        )
    if 'EPS_Growth_NTM' in peers_display.columns:
        peers_display['EPS_Growth_NTM'] = peers_display['EPS_Growth_NTM'].apply(
            lambda x: f"{x:.1f}%" if pd.notna(x) else 'N/A'
        )
    if 'Gross_Margin' in peers_display.columns:
        peers_display['Gross_Margin'] = peers_display['Gross_Margin'].apply(
            lambda x: f"{x:.1f}%" if pd.notna(x) else 'N/A'
        )
    
    print(f"\n{dimension_name} Peers:")
    display(peers_display)

# Display peer details for each dimension
print(f"\n{'='*80}")
print("PEER DETAIL TABLES")
print(f"{'='*80}")

display_peer_details(regression_results.get('PE_2Y_Sector', {}), "Sector")
display_peer_details(regression_results.get('PE_2Y_Growth', {}), "Growth Cohort")
display_peer_details(regression_results.get('PE_2Y_Profitability', {}), "Profitability")

In [ ]:
# Cell 6.3: Implied Valuation Summary

def calculate_implied_valuations(regression_results, projections_df, target_data):
    """
    Calculate implied price targets and IRR based on regression results and projections.
    """
    
    current_price = target_data.get('Price', 0)
    
    results = []
    
    # Get EPS projections for Year 2 and Year 5
    if len(projections_df) >= 2:
        y2_eps = projections_df['EPS'].iloc[1] if 'EPS' in projections_df.columns else None
    else:
        y2_eps = None
    
    if len(projections_df) >= 5:
        y5_eps = projections_df['EPS'].iloc[4] if 'EPS' in projections_df.columns else None
    else:
        y5_eps = None
    
    # 2-Year implied prices (average of dimensions)
    pe_2y_results = []
    for key in ['PE_2Y_Sector', 'PE_2Y_Growth', 'PE_2Y_Profitability']:
        result = regression_results.get(key, {})
        if result.get('success'):
            pe_2y_results.append({
                'multiple': result['implied_value'],
                'ci_lower': result['ci_lower'],
                'ci_upper': result['ci_upper']
            })
    
    if pe_2y_results and y2_eps:
        avg_2y_pe = np.mean([r['multiple'] for r in pe_2y_results])
        avg_2y_pe_low = np.mean([r['ci_lower'] for r in pe_2y_results])
        avg_2y_pe_high = np.mean([r['ci_upper'] for r in pe_2y_results])
        
        implied_2y_price = avg_2y_pe * y2_eps
        implied_2y_price_low = avg_2y_pe_low * y2_eps
        implied_2y_price_high = avg_2y_pe_high * y2_eps
        
        # Calculate 2-year IRR
        if current_price > 0:
            irr_2y = ((implied_2y_price / current_price) ** (1/2) - 1) * 100
            irr_2y_low = ((implied_2y_price_low / current_price) ** (1/2) - 1) * 100
            irr_2y_high = ((implied_2y_price_high / current_price) ** (1/2) - 1) * 100
        else:
            irr_2y = irr_2y_low = irr_2y_high = None
        
        results.append({
            'Horizon': 'Year 2',
            'Projected EPS': f"${y2_eps:.2f}",
            'Avg Implied P/E': f"{avg_2y_pe:.1f}x",
            'Implied Price': f"${implied_2y_price:.2f}",
            'Price Range': f"${implied_2y_price_low:.2f} - ${implied_2y_price_high:.2f}",
            'vs Current': f"{((implied_2y_price/current_price)-1)*100:+.1f}%" if current_price > 0 else 'N/A',
            'Implied IRR': f"{irr_2y:.1f}%" if irr_2y else 'N/A'
        })
    
    # 5-Year terminal implied price
    pe_5y_result = regression_results.get('PE_5Y_Terminal', {})
    if pe_5y_result.get('success') and y5_eps:
        terminal_pe = pe_5y_result['implied_value']
        terminal_pe_low = pe_5y_result['ci_lower']
        terminal_pe_high = pe_5y_result['ci_upper']
        
        implied_5y_price = terminal_pe * y5_eps
        implied_5y_price_low = terminal_pe_low * y5_eps
        implied_5y_price_high = terminal_pe_high * y5_eps
        
        # Calculate 5-year IRR
        if current_price > 0:
            irr_5y = ((implied_5y_price / current_price) ** (1/5) - 1) * 100
            irr_5y_low = ((implied_5y_price_low / current_price) ** (1/5) - 1) * 100
            irr_5y_high = ((implied_5y_price_high / current_price) ** (1/5) - 1) * 100
        else:
            irr_5y = irr_5y_low = irr_5y_high = None
        
        results.append({
            'Horizon': 'Year 5 (Terminal)',
            'Projected EPS': f"${y5_eps:.2f}",
            'Avg Implied P/E': f"{terminal_pe:.1f}x",
            'Implied Price': f"${implied_5y_price:.2f}",
            'Price Range': f"${implied_5y_price_low:.2f} - ${implied_5y_price_high:.2f}",
            'vs Current': f"{((implied_5y_price/current_price)-1)*100:+.1f}%" if current_price > 0 else 'N/A',
            'Implied IRR': f"{irr_5y:.1f}%" if irr_5y else 'N/A'
        })
    
    return pd.DataFrame(results)


# Calculate and display implied valuations
print(f"\n{'='*80}")
print("IMPLIED VALUATION SUMMARY")
print(f"{'='*80}\n")

print(f"Current Price: ${target_data.get('Price', 0):.2f}")
print(f"Current P/E (NTM): {target_data.get('PE_NTM', 'N/A'):.1f}x\n")

valuation_summary = calculate_implied_valuations(regression_results, projections_df, target_data)
if not valuation_summary.empty:
    display(valuation_summary.style.set_properties(**{'text-align': 'center'}))
else:
    print("Unable to calculate implied valuations - check projections data")

---

## Section 7: Validation & Warnings

In [ ]:
# Cell 7.1: Validation & Warnings Summary

def run_all_validations(regression_results, target_data, projections_df):
    """
    Run comprehensive validation checks and generate warnings.
    """
    
    warnings = []
    info = []
    
    # Check target company data
    if target_data.get('PE_NTM', 0) <= 0:
        warnings.append("Target has negative or zero P/E - auto-switching to P/FCF focus recommended")
    
    eps_growth = target_data.get('EPS_Growth_NTM', None)
    if eps_growth is not None and (eps_growth < -20 or eps_growth > 50):
        warnings.append(f"Target growth ({eps_growth:.1f}%) is extreme - extrapolation likely")
    
    # Check projections
    if projections_df.empty or len(projections_df) < 5:
        warnings.append("Incomplete 5-year projections - some analyses may be unavailable")
    
    # Check regression results
    successful_regressions = sum(1 for k, v in regression_results.items() if v.get('success', False))
    total_regressions = len([k for k in regression_results.keys() if 'PE' in k or 'PFCF' in k])
    
    if successful_regressions == 0:
        warnings.append("CRITICAL: No successful regressions - check data availability")
    elif successful_regressions < total_regressions / 2:
        warnings.append(f"Only {successful_regressions}/{total_regressions} regressions successful")
    
    # Check individual regressions
    for key, result in regression_results.items():
        if not result.get('success'):
            continue
        
        dim_name = key.replace('PE_', '').replace('PFCF_', '').replace('_', ' ')
        
        # Peer count check
        if result['peer_count'] < MIN_PEERS:
            warnings.append(f"{dim_name}: Small peer set ({result['peer_count']} peers) - consider widening criteria")
        
        # R-squared check
        if result['r_squared'] < 0.3:
            warnings.append(f"{dim_name}: Very weak fit (R²={result['r_squared']:.2f}) - use with extreme caution")
        elif result['r_squared'] < 0.5:
            info.append(f"{dim_name}: Moderate fit (R²={result['r_squared']:.2f}) - interpret with caution")
        
        # Extrapolation check
        if result.get('is_extrapolation', False):
            x_min, x_max = result['x_range']
            warnings.append(f"{dim_name}: Target outside peer range ({x_min:.1f}% - {x_max:.1f}%) - extrapolating")
        
        # Confidence interval width check
        ci_width = result['ci_upper'] - result['ci_lower']
        ci_pct = ci_width / result['implied_value'] * 100
        if ci_pct > 50:
            info.append(f"{dim_name}: Wide confidence interval ({ci_pct:.0f}% of implied multiple)")
    
    return warnings, info


# Run validations
print(f"\n{'='*80}")
print("VALIDATION & WARNINGS")
print(f"{'='*80}\n")

warnings, info = run_all_validations(regression_results, target_data, projections_df)

if warnings:
    print("WARNINGS:")
    for w in warnings:
        print(f"  ⚠️  {w}")
else:
    print("  ✓ No critical warnings")

if info:
    print("\nINFORMATIONAL NOTES:")
    for i in info:
        print(f"  ℹ️  {i}")

# Summary statistics
print(f"\n{'='*80}")
print("ANALYSIS SUMMARY")
print(f"{'='*80}\n")

successful = [k for k, v in regression_results.items() if v.get('success', False)]
print(f"Successful Regressions: {len(successful)}/{len(regression_results)}")
print(f"  P/E Regressions: {len([k for k in successful if 'PE' in k])}")
print(f"  P/FCF Regressions: {len([k for k in successful if 'PFCF' in k])}")

# Average R-squared
r_squared_vals = [v['r_squared'] for v in regression_results.values() if v.get('success')]
if r_squared_vals:
    print(f"\nAverage R²: {np.mean(r_squared_vals):.2f}")
    print(f"Best R²: {np.max(r_squared_vals):.2f}")
    print(f"Lowest R²: {np.min(r_squared_vals):.2f}")

In [ ]:
# Cell 7.2: Interactive Peer Exclusion (Re-run Analysis)

# =============================================================================
# MANUAL PEER EXCLUSION
# Add tickers to exclude from the analysis, then re-run this cell
# =============================================================================

EXCLUDED_TICKERS = [
    # Add tickers to exclude here, e.g.:
    # "MSFT",
    # "GOOGL",
]

def rerun_analysis_with_exclusions(excluded_tickers):
    """
    Re-run the regression analysis with specified peers excluded.
    """
    
    if not excluded_tickers:
        print("No tickers excluded. To exclude peers, add tickers to EXCLUDED_TICKERS list above.")
        return None
    
    print(f"Excluding {len(excluded_tickers)} peers: {', '.join(excluded_tickers)}")
    print("="*60)
    
    # Filter out excluded tickers from each peer set
    def filter_excluded(df, excluded):
        if df.empty or 'Ticker' not in df.columns:
            return df
        mask = ~df['Ticker'].str.upper().isin([t.upper() for t in excluded])
        return df[mask].copy()
    
    # Re-run regressions with filtered peers
    updated_results = {}
    
    # 2-Year P/E
    for dim_name, peers_df_orig in [('Sector', sector_peers), ('Growth', growth_peers), ('Profitability', profit_peers)]:
        peers_filtered = filter_excluded(peers_df_orig, excluded_tickers)
        
        if not peers_filtered.empty:
            result = run_regression(
                df=peers_filtered,
                x_col='EPS_Growth_NTM',
                y_col='PE_NTM',
                target_x_value=target_eps_growth
            )
            updated_results[f'PE_2Y_{dim_name}'] = result
            
            if result['success']:
                print(f"{dim_name} P/E: {result['implied_value']:.1f}x (was {regression_results.get(f'PE_2Y_{dim_name}', {}).get('implied_value', 'N/A')})")
    
    return updated_results

# Run with exclusions if any are specified
if EXCLUDED_TICKERS:
    updated_regression_results = rerun_analysis_with_exclusions(EXCLUDED_TICKERS)
    
    if updated_regression_results:
        print("\nUpdated results stored in 'updated_regression_results' variable.")
        print("To visualize, update the regression_results dict and re-run visualization cells.")
else:
    print("No peers excluded. Analysis results unchanged.")

---

## Usage Notes

### Quick Start
1. **Enter Target Ticker**: Update `TARGET_TICKER` in Cell 1.3
2. **Paste Projections**: Update `PROJECTIONS_TSV` with your 5-year projections (tab-separated)
3. **Run All Cells**: Execute all cells in order (Kernel → Run All)

### Configuration Options
- **GICS Level**: Change `GICS_LEVEL` to `'Sector'`, `'Industry_Group'`, `'Industry'`, or `'Sub_Industry'`
- **Market Cap Range**: Adjust `MCAP_RANGE_LOW` and `MCAP_RANGE_HIGH`
- **Growth Tolerance**: Modify `GROWTH_TOLERANCE` for growth cohort selection
- **Peer Counts**: Adjust `MIN_PEERS` and `MAX_PEERS`

### Excluding Peers
1. Identify outlier peers from the charts (hover for ticker)
2. Add tickers to `EXCLUDED_TICKERS` list in Cell 7.2
3. Re-run Cell 7.2 to update results

### Interpreting Results
- **R² > 0.6**: Strong relationship, high confidence in results
- **R² 0.4-0.6**: Moderate relationship, interpret with caution
- **R² < 0.4**: Weak relationship, consider alternative peer sets

### Known Limitations
- Requires positive NTM earnings for P/E analysis
- Cross-sector peers may have structural differences
- Historical averages may not reflect future expectations

---

*Valuation Agent v1.0 - Algorithmic Comparable Analysis Tool*